## Check enviroment is updated

In [1]:
conda_list = !conda list
conda_env_string = "".join(conda_list)
def check_installed_requirement(reqs):
    for req in reqs:
        if req in conda_env_string:
            print(f"requirement {req} has been installed")
        else:
            print(f"installing {req} by using conda ...")
            !conda install {req}

In [2]:
check_installed_requirement(["spotipy","alive-progress","tqdm","yaml"])

requirement spotipy has been installed
requirement alive-progress has been installed


In [3]:
%%capture
from tqdm.auto import tqdm as tqdm
tqdm().pandas()
from colorama import Fore

In [4]:
# Load config
import yaml

In [5]:
# Làm những cái khác
import os
import pandas as pd
import csv
import time
import re

In [6]:
# Của spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [7]:
from math import ceil

In [8]:
# In lỗi trong quá trình crawling
import traceback

In [9]:
# Ngày tháng
from datetime import datetime

## Load config

In [10]:
# Kiểm tra file config có tồn tại
if not os.path.isfile("config.yaml"):
    sys.exit("'config.yaml' not found! Please add it and try again.")
else:
    with open("config.yaml") as file:
        config = yaml.load(file, Loader=yaml.FullLoader)

In [11]:
# Cài đặt biến môi trường

os.environ['SPOTIPY_CLIENT_ID']= config['SPOTIPY_CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET']= config['SPOTIPY_CLIENT_SECRET']

In [12]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

## Test API Client

In [13]:
playlists = sp.user_playlists('spotify',offset=1390,limit=10)

In [14]:
len(playlists['items'])

5

In [15]:
playlists['items'][0]

{'collaborative': False,
 'description': 'Kick back with the queens of pop.',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX3WvGXE8FqYX'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX3WvGXE8FqYX',
 'id': '37i9dQZF1DX3WvGXE8FqYX',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706f0000000361dceefe400614407dd0177f',
   'width': None}],
 'name': 'Women of Pop',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MTYyOTgxNTMwMiwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX3WvGXE8FqYX/tracks',
  'total': 70},
 'type': 'playlist',
 'uri': 'spotify:playlist:37i9dQZF1DX3WvGXE8FqYX'}

In [16]:
playlists['next']

In [17]:
playlists['items'][0]

{'collaborative': False,
 'description': 'Kick back with the queens of pop.',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX3WvGXE8FqYX'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX3WvGXE8FqYX',
 'id': '37i9dQZF1DX3WvGXE8FqYX',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706f0000000361dceefe400614407dd0177f',
   'width': None}],
 'name': 'Women of Pop',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MTYyOTgxNTMwMiwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX3WvGXE8FqYX/tracks',
  'total': 70},
 'type': 'playlist',
 'uri': 'spotify:playlist:37i9dQZF1DX3WvGXE8FqYX'}

In [18]:
tracks = sp.playlist(playlists['items'][0]['id'])
# tracks

Như vậy mỗi lần ta sẽ get được `limit` playlist - một playlist trả ra một list các track và tổng số track, ngoài ra còn có trả về một url khác để ta có thể crawl segment tiếp theo. Theo như ta test chặn đầu cuối thì sẽ có tổng cộng 1395 playlist.

### tqdm test

In [19]:
sleep_time = 0.1

In [20]:
for i in tqdm(range(10),desc=f"Test tqdm",leave=True):
    time.sleep(sleep_time)

Test tqdm:   0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
for i in tqdm(range(10), position = 0, leave = True):
    for j in tqdm(range(5), position = 1, leave = False):
        time.sleep(sleep_time)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

## Define Constant

In [22]:
date = str(datetime.today().strftime('%Y%m%d_%H%M%S'))
date

'20210824_212829'

In [23]:
playlists_path = f"../data/{date}_playlists.tsv"
tracks_path = f"../data/{date}_tracks.tsv"
audios_path = f"../data/{date}_audios.tsv"
artist_path = f"../data/{date}_artists.tsv"

## Crawl Playlist

Okay, giờ crawl playlist với cái thanh màu mè phía trên - 1395 playlists

In [24]:
# Chạy một offset từ 0 --> 1 (tương đương với 0 --> 50), như vậy từ 0 --> i sẽ tương đương với 0 --> 1395~(1400) --> i = 28
# Hay nói đơn giản hơn, để crawl k playlist ta cần chạy offset từ 0 --> ceil(k/limit)

def crawl_playlist(nplaylist=50,limit=50,file_to_save="data/playlists.tsv",override=False):
#     tính số segment cần crawl
    nsegment = ceil(nplaylist/limit)
    playlist_data = {}
    playlist_cols = [
        'playlist_id',
        'playlist_name',
        'playlist_description',
        'playlist_num_tracks',
        'playlist_num_followers'
    ]
    
    if override:
        with open(file_to_save, 'w') as f:
            f.write('\t'.join(playlist_cols) + '\n')
    
    # print(f"Crawling {nsegment} segments with limit={limit}, nplaylist={nplaylist}")
    for nseg in tqdm(range(0, nplaylist , limit),desc=f"Crawling {nsegment} segments with limit={limit}, nplaylist={nplaylist}",ascii=True):
        time.sleep(sleep_time)
        # print(f"Crawling segment from: {nseg + 1} to {nseg + limit + 1} ...")
        playlists = sp.user_playlists(user='spotify', offset=nseg,limit=limit)

        playlist_data = {}
        for col in playlist_cols:
            playlist_data[col] = []

        for __playlist in tqdm(playlists['items'],leave=False,position=1, unit_scale=True,desc=f"Crawling segment {ceil(nseg/limit)} from: {nseg + 1} to {nseg + limit + 1}"):
            try:
                playlist = sp.playlist(__playlist['id'])

                # Get playlist ID from PlaylistObject
                playlist_data['playlist_id'].append(playlist['id'])
                playlist_data['playlist_name'].append(playlist['name'])
                playlist_data['playlist_description'].append(playlist['description'])
                
                # Get data from PlaylistTracksRefObject
                playlist_data['playlist_num_tracks'].append(playlist['tracks']['total'])
                
                
                # Get data from FollowersObject
                playlist_data['playlist_num_followers'].append(playlist['followers']['total'])

            except Exception:
                print(traceback.format_exc())
                continue
                
                
        with open(file_to_save, 'a',encoding='utf-8') as f:
            for i in range(len(playlist_data['playlist_id'])):
                write_string = '\t'.join([re.sub(r'(\n+|\t+|\r+)(?=[A-Z])', ' ', str(playlist_data[k][i]).strip()) for k in playlist_cols]).encode().decode("utf-8")
                if write_string:
                    f.write(write_string + '\n')

In [25]:
crawl_playlist(file_to_save=playlists_path,nplaylist=1500,limit=50,override=True)

Crawling 30 segments with limit=50, nplaylist=1500:   0%|          | 0/30 [00:00<?, ?it/s]

Crawling segment 0 from: 1 to 51:   0%|          | 0.00/50.0 [00:00<?, ?it/s]

Crawling segment 1 from: 51 to 101:   0%|          | 0.00/50.0 [00:00<?, ?it/s]

Crawling segment 2 from: 101 to 151:   0%|          | 0.00/50.0 [00:00<?, ?it/s]

Crawling segment 3 from: 151 to 201:   0%|          | 0.00/50.0 [00:00<?, ?it/s]

...

Crawling segment 25 from: 1251 to 1301:   0%|          | 0.00/50.0 [00:00<?, ?it/s]

Crawling segment 26 from: 1301 to 1351:   0%|          | 0.00/50.0 [00:00<?, ?it/s]

Crawling segment 27 from: 1351 to 1401:   0%|          | 0.00/45.0 [00:00<?, ?it/s]

Crawling segment 28 from: 1401 to 1451: 0.00it [00:00, ?it/s]

Crawling segment 29 from: 1451 to 1501: 0.00it [00:00, ?it/s]

## Crawl Tracks

In [26]:
df = pd.read_csv(playlists_path,sep="\t")
playlist_ids = df["playlist_id"]
print(playlist_ids[0:5])

0    37i9dQZF1DXcBWIGoYBM5M
1    37i9dQZF1DX0XUsuxWHRQd
2    37i9dQZF1DX1lVhptIYRda
3    37i9dQZF1DX10zKzsJ2jva
4    37i9dQZF1DX4JAvHpjipBk
Name: playlist_id, dtype: object


In [27]:
def crawl_tracks(playlist_ids,file_to_save="data/tracks.tsv",override=False):
    __playlist_id = playlist_ids
    
    
    track_data = {}
    track_cols = [
        'playlist_id',
        'track_id',
        'track_name',
        'track_duration',
        'artist_ids',
        'artist_names',
        'album_id',
        'album_name',
#         'available_markets',
        'popularity',
    ]
    
    
    if override:
        with open(file_to_save, 'w') as f:
            f.write('\t'.join(track_cols) + '\n')
    
    for __id in tqdm(__playlist_id,desc=f"Crawling total {len(__playlist_id)} playlists track data",ascii=True):
        time.sleep(sleep_time)
        try:
            playlist = sp.playlist(__id)

            track_data = {}
            for col in track_cols:
                track_data[col] = []

            for _track in playlist['tracks']['items']:
                
                track = _track['track']
                
                if not track:
                    continue
                track_data['playlist_id'].append(__id)
                
                track_data['track_id'].append(track['id'])
                track_data['track_name'].append(track['name'])
                track_data['track_duration'].append(track['duration_ms'])

    #             track_data['available_markets'].append(track['available_markets'])
                track_data['popularity'].append(track['popularity'])

                track_data['artist_ids'].append([artist['id'] for artist in track['artists']])
                track_data['artist_names'].append([artist['name'] for artist in track['artists']])


                track_data['album_id'].append(track['album']['id'])
                track_data['album_name'].append(track['album']['name'])
        except Exception:
            print(f"Error in playlist id {__id}; link to playlist {playlist['external_urls']['spotify']}")
            print(traceback.format_exc())
            print(track_data)
            continue

        with open(file_to_save, 'a',encoding='utf-8') as f:
            for i in range(len(track_data['track_id'])):
                write_string = '\t'.join([re.sub(r'(\n+|\t+|\r+)(?=[A-Z])',' ', str(track_data[k][i]).strip()) for k in track_cols]).encode().decode("utf-8")
                if write_string:
                    f.write(write_string + '\n')

In [29]:
crawl_tracks(playlist_ids,file_to_save=tracks_path,override=True)

Crawling total 1395 playlists track data:   0%|          | 0/1395 [00:00<?, ?it/s]

In [30]:
# pl = sp.playlist('37i9dQZF1DWXMg4uP5o3dm')
# pl['tracks']['items'][18] #['track']['id']

In [31]:
# pl = sp.playlist('37i9dQZF1DWU2LcZVHsTdv')
# '\t'.join([re.sub(, ' ', str(pl['description']).strip())]).encode().decode("utf-8")

## Crawl Audio Features

In [32]:
df = pd.read_csv(tracks_path,sep="\t")
# playlist_ids = df["playlist_id"]
# print(playlist_ids[0:5])
df['track_id']

0        5HCyWlXZPP0y6Gqq8TgA20
1        27NovPIUIRrOZoCHxABJwK
2        3Vi5XqYrmQgOYBajMWSvCi
3        6PQ88X9TkUIAUIZJHW2upE
4        3Wrjm47oTz2sjIgck11l5e
                  ...          
88814    5kqIPrATaCc2LqxVWzQGbk
88815    3FCto7hnn1shUyZL42YgfO
88816    3XVBdLihbNbxUwZosxcGuJ
88817    6QPKYGnAW9QozVz2dSWqRg
88818    1VdZ0vKfR5jneCmWIUAMxK
Name: track_id, Length: 88819, dtype: object

In [33]:
track_ids = df['track_id'].drop_duplicates()

In [34]:
def crawl_audios(track_ids,file_to_save="data/audios.tsv",override=False,limit=50):
    __track_ids = track_ids
    audio_cols = [
        'id',
        'danceability',
        'energy',
        'key',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo',
        'time_signature'
    ]

    if override:
        with open(file_to_save, 'w') as f:
            f.write('\t'.join(audio_cols) + '\n')
    
    for start in tqdm(range(0, len(__track_ids), limit), desc=f"Crawling total {len(__track_ids)} audio features data",ascii=True):
        time.sleep(sleep_time)
        
        audio_data = {}
        for col in audio_cols:
            audio_data[col] = []
        
        end = min(start+limit, len(track_ids))
        
        audio_features = sp.audio_features(track_ids[start:end])
        
        for audio in (audio_features):
            if not audio:
                continue
            try:
                for cols in audio_cols:
                    audio_data[cols].append(audio[cols])

            except Exception:
                print(f"Error in track id {audio}")
                print(traceback.format_exc())
                continue
                
        with open(file_to_save, 'a',encoding='utf-8') as f:
            for i in range(len(audio_data['id'])):
                write_string = '\t'.join([re.sub(r'(\n+|\t+|\r+)(?=[A-Z])',' ', str(audio_data[k][i]).strip()) for k in audio_cols]).encode().decode("utf-8")
                if write_string:
                    f.write(write_string + '\n')

In [36]:
crawl_audios(track_ids,file_to_save=audios_path,override=True,limit=50)

Crawling total 70031 audio features data:   0%|          | 0/1401 [00:00<?, ?it/s]

## Crawl Artist

In [37]:
df = pd.read_csv(tracks_path,sep="\t")
artist_ids = df['artist_ids'].apply(lambda x: x[1:-1].split(", ")).explode().apply(lambda x: x[1:-1]).to_list()
artist_ids = list(set(artist_ids))

In [38]:
def crawl_artist(artist_ids,file_to_save="data/artists.tsv",override=False,limit=50):

    __artist_ids = artist_ids
    artist_cols = [
        'id',
        'name',
        'followers',
        'popularity',
        'genres',
    ]
    
    if override:
        with open(file_to_save, 'w') as f:
            f.write('\t'.join(artist_cols) + '\n')

    for start in tqdm(range(0, len(__artist_ids), limit),  desc=f"Crawling total {len(__artist_ids)} artists data",ascii=True):
        time.sleep(sleep_time)

        artist_data = {}
        for col in artist_cols:
            artist_data[col] = []

        end = min(start+limit, len(artist_ids))
        artist_features = sp.artists(artist_ids[start:end])
        for artist in artist_features['artists']:
            try:
                if not artist:
                    continue
                artist_data['followers'].append(artist['followers']['total'])
                artist_data['id'].append(artist['id'])
                artist_data['name'].append(artist['name'])
                artist_data['popularity'].append(artist['popularity'])
                artist_data['genres'].append(artist['genres'])
            except Exception:
                print(f"Error in artist id {artist}")
                print(traceback.format_exc())
                continue
        with open(file_to_save, 'a',encoding='utf-8') as f:
            for i in range(len(artist_data['id'])):
                write_string = '\t'.join([re.sub(r'(\n+|\t+|\r+)(?=[A-Z])',' ', str(artist_data[k][i]).strip()) for k in artist_cols]).encode().decode("utf-8")
                if write_string:
                    f.write(write_string + '\n')

In [41]:
crawl_artist(artist_ids,file_to_save=artist_path,override=True)

Crawling total 34655 artists data:   0%|          | 0/694 [00:00<?, ?it/s]